In [19]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

In [20]:
def parse_task(task, prefix):
    try:
        if re.search(prefix, task.name):
            # print (task.id + '\t' + task.name + '\t' + str(task.price.amount) + '\n')
            metrics = task.outputs['RNASeQC_Metrics'].content()
            data = metrics.split('\n')
            end1 = data[23].split('\t')
            end2 = data[24].split('\t')
            # pdb.set_trace()
            return task.id + '\t' + task.name + '\t' + end1[1] + '\t' + end2[1] + '\n'
    except Exception as e:
        sys.stderr.write('Could not process task ' + task.id + ' ' + task.name + ' got error ' + str(e) + '\n')
        sys.exit()

In [21]:
id_pre = 'RNAfusion'
suffix = '-Reharmonization'
project = 'kfdrc-harmonization/sd-bhjxbdqk-06'

#test = api.files.get(id='5c60b3b4e4b0c5cd89bde931')

tasks = api.tasks.query(project=project, status='COMPLETED').all()

out_fh = open('/Users/brownm28/Documents/2019-Jan-31_rna_fusion/strand_info/strand_info.txt', 'w')
out_fh.write('Task ID\tTask name\tend 1 sense rate\tend2 sense rate\n')

with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(parse_task, task, id_pre): task for task in tasks}
    for strand_info in concurrent.futures.as_completed(results):
        try:
            if strand_info.result() is not None:
                out_fh.write(strand_info.result())
        except Exception as e:
            print (e)
out_fh.close()

#for task in tasks:
#    parse_task(task, id_pre)